In [270]:
import pandas as pd
import numpy as np
import missingno as msno
%matplotlib inline

In [271]:
path_train = '/home/alex/Documents/Kaggle/Loans/train.csv'
path_test = '/home/alex/Documents/Kaggle/Loans/test.csv'
train = pd.read_csv(path_train)
test = pd.read_csv(path_test)

In [272]:
train.head()

,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status
0,LP001002,Male,No,0,Graduate,No,5849,0.0,NaN,360.0,1.0,Urban,Y
1,LP001003,Male,Yes,1,Graduate,No,4583,1508.0,128.0,360.0,1.0,Rural,N
2,LP001005,Male,Yes,0,Graduate,Yes,3000,0.0,66.0,360.0,1.0,Urban,Y
3,LP001006,Male,Yes,0,Not Graduate,No,2583,2358.0,120.0,360.0,1.0,Urban,Y
4,LP001008,Male,No,0,Graduate,No,6000,0.0,141.0,360.0,1.0,Urban,Y


In [273]:
train.shape

(614, 13)

In [274]:
test.shape

(367, 12)

In [275]:
columns = ['Gender', 'Married', 'Education', 'Self_Employed', 'Property_Area', 'Dependents']

In [276]:
print('Missing train')
train.isna().sum()

Missing train


Loan_ID               0
Gender               13
Married               3
Dependents           15
Education             0
Self_Employed        32
ApplicantIncome       0
CoapplicantIncome     0
LoanAmount           22
Loan_Amount_Term     14
Credit_History       50
Property_Area         0
Loan_Status           0
dtype: int64

In [277]:
print('Missing test')
test.isna().sum()

Missing test


Loan_ID               0
Gender               11
Married               0
Dependents           10
Education             0
Self_Employed        23
ApplicantIncome       0
CoapplicantIncome     0
LoanAmount            5
Loan_Amount_Term      6
Credit_History       29
Property_Area         0
dtype: int64

In [278]:
train.Dependents.describe()

count     599
unique      4
top         0
freq      345
Name: Dependents, dtype: object

In [279]:
train.Gender.fillna('Male', inplace=True)
test.Gender.fillna('Male', inplace=True)

train.Married.fillna('Yes', inplace=True)
test.Married.fillna('Yes', inplace=True)

train.Dependents.fillna(0, inplace=True)
test.Dependents.fillna(0, inplace=True)

train.Self_Employed.fillna('No', inplace=True)
test.Self_Employed.fillna('No', inplace=True)

train.LoanAmount.fillna(146, inplace=True)
test.LoanAmount.fillna(136, inplace=True)

train.Loan_Amount_Term.fillna(342, inplace=True)
test.Loan_Amount_Term.fillna(342, inplace=True)

train.Credit_History.fillna(0.84, inplace=True)
test.Credit_History.fillna(0.82, inplace=True)

In [280]:
train_df = pd.get_dummies(train, columns=columns)


In [281]:
test_df = pd.get_dummies(test, columns=columns)


In [282]:
train_df.set_index('Loan_ID', inplace=True)
test_df.set_index('Loan_ID', inplace=True)

In [283]:
mapping = {'Y': 1, 'N': 0}
train_df.Loan_Status  = [mapping[item] for item in train_df.Loan_Status]

In [286]:
X = train_df.drop(['Loan_Status'], axis=1)
y = train_df['Loan_Status']
X_test = test_df

In [287]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [291]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import mean_absolute_error

forest = RandomForestClassifier(max_depth=10, random_state=42)
forest.fit(X_train, y_train)
forest_pred = forest.predict(X_test)

In [292]:
mean_absolute_error(y_test, forest_pred)

0.19704433497536947

In [307]:
forest.fit(X, y)
forest_pred = forest.predict(test_df)

In [310]:
submission = pd.DataFrame(test['Loan_ID'])
mapping = {1: 'Y', 0: 'N'}
forest_pred = [mapping[item] for item in forest_pred]
submission['Loan_Status'] = forest_pred

In [313]:
submission.to_csv('my_submission.csv', index=False)